Import Libraries
TO DO
1) Describe Grid Location bottom left hand location and Cell Size
2) Plot the vector using x y coordinates.
3) Compare to Raster Stat Results, need to make sure that it only includes the cells fully inside (not on the border of the vector.

In [18]:
import numpy as np
print (np.__version__)
from osgeo import gdal
from scipy import stats
import timeit
from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon
from osgeo import ogr
from math import ceil,floor

1.11.3


<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

Load test data

In [90]:
r = gdal.Open("data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif")
shape = "data\\test.shp"
g = np.array(r.GetRasterBand(1).ReadAsArray())
csze = r.GetGeoTransform()[1]
ulX = r.GetGeoTransform(0)
ulY = r.GetGeoTransform(3)
print (uly)

(1599633.055684202, 30.0, 0.0, 2083307.9820680085, 0.0, -30.0)


In [5]:
## I checked the top left cell in q gis and it is value = 95 bottom left = 41. I want g[0,0] = 41 so that the bottom left is 0,0
##print(g[0,0])
##g = np.flipud(g)
##print(g[0,0])

In [6]:
##stats.describe(g) 
print( (r.RasterYSize,r.RasterXSize))
print( g.shape )
##out.shape

(1734, 2501)
(1734, 2501)


Function to create data structure function takes in a grid with mxn and returns a grid of the same size summed by columns.  A vectorized version would be faster

In [7]:
def dtFZS(g,out):
    for xw in range(0,g.shape[1]):
        for yw in range(1,g.shape[0]):
            out[yw,xw] = out[yw-1,xw] + g[yw,xw]
    print("done")

In [8]:
##out = dtFZS(g)
DS_FstZS = np.zeros(g.shape,dtype='uint64')
dtFZS(g,DS_FstZS)


done


In [9]:
##stats.describe(DS_FstZS)
##out.shape

Fast Summary Statistics Function
    

In [59]:
## Create a function that takes in x1, x2, ulX, and cllsze in world coords and returns list of x locations in grid coordinates
def x_world_to_grid(wx1,wx2,ulX,clSze):
    x1a = min(wx1,wx2)
    x2a = max(wx1,wx2)
    
    xstart = ceil  ((x1a - ulX - (clSze/2)) / clSze)
    xstop =  floor ((x2a - ulX - (clSze/2)) / clSze)
    return(list(range(xstart,xstop)))

x_world_to_grid(510,560,500,10)



[1, 2, 3, 4]

In [61]:
## Create a Function that returns the centroid locations in world cordinates
## accepts a list as input

def x_grid_to_world(gx,ulX,clSze):
    return [i * clSze + ulX + (clSze/2) for i in gx]
    
x_grid_to_world(x_world_to_grid(510,560,500,10),500,10)    
    

[515.0, 525.0, 535.0, 545.0]

In [63]:
## Create a function that takes in world coordinates of x_pred x1,x2,y1,y2 and returns y_pred as a list
def y_world_pred(wx1,wx2,wy1,wy2,wxpred):
    if (wx1 > wx2): ## switch Coords
        wx1,wx2,wy1,wy2 = wx2,wx1,wy2,wy1
    slope = (wy2-wy1)/(wx2-wx1)
    ##return [ i for i in xpred]
    return [(slope * (i-wx1)) + wy1 for i in wxpred]

y_world_pred(510,560,3,7,x_grid_to_world(x_world_to_grid(510,560,500,10),500,10))

[3.4, 4.2, 5.0, 5.800000000000001]

In [77]:
## Snap world y to grid coordinates wy is a list of cords
def y_world_to_grid(ulY,clSze,wypred):
        return [ceil(i - ulY - (clSze/2) / clSze ) for i in wypred]

In [81]:
## Examples on how to use the world to grid grid to world helper functions

## def x_world_to_grid(wx1,wx2,ulX,clSze):
wx1 = 560; wx2=510; wy1 = 10; wy2 = 3; ulX = 500; ulY = 0; clSze=10;

xgrid  = x_world_to_grid(wx1,wx2,ulX,clSze)
print(xgrid)
wxpred = x_grid_to_world(xgrid,ulX,clSze)  
print(wxpred)
wypred = y_world_pred(wx1,wx2,wy1,wy2,wxpred)
print(wypred)
ygrid = y_world_to_grid(ulY,clSze,wypred)
print(ygrid)

[1, 2, 3, 4]
[515.0, 525.0, 535.0, 545.0]
[3.7, 5.1, 6.5, 7.9]
[4, 5, 6, 8]


In [118]:
## Combine all the helper functions to return list of cells to sum

wx1 = 560; wx2=510; wy1 = 10; wy2 = 3; ulX = 500; ulY = 0; clSze=10;
def xy_grid_cnt(wx1,wx2,ulX,wy1,wy2,ulY,clSze):
    xgrid  = x_world_to_grid(wx1,wx2,ulX,clSze)
    wxpred = x_grid_to_world(xgrid,ulX,clSze)  
    wypred = y_world_pred(wx1,wx2,wy1,wy2,wxpred)
    ygrid = y_world_to_grid(ulY,clSze,wypred)
    return(list ( zip(xgrid,ygrid)))
    
xy_grid_cnt(wx1,wx2,ulX,wy1,wy2,ulY,clSze)    






[(1, 4), (2, 5), (3, 6), (4, 8)]

In [137]:
##  To DO
## This function needs to be rewritten to make v a lattice on the grid (not matching the grid locations but between the grid valeus)
## and to take coordinates

def FstZS(v,DS_FstZS,ulX,ulY,clSze):
    total = 0;    
    for p in range(1,(len(v)+0)):
        ## If moving left
        if v[p][1] > v[p-1][1]: 
            print (1)
        elif v[p][1] > v[p-1][1]:
            cords = xy_grid_cnt(v[p][0],v[p-1][0],ulX,v[p][1],v[p-1][1],ulY,clSze)
            print (cords)
            ##total += np.sum(DS_FstZS[xy_grid_cnt(v[p][0],v[p-1][0],ulX,v[p][1],v[p-1][1],ulY,clSze)])
        ## If moving Right
        elif v[p][1] < v[p-1][1]: 
            cords = xy_grid_cnt(v[p][0],v[p-1][0],ulX,v[p][1],v[p-1][1],ulY,clSze)
            print (cords)  
             ##total += np.sum(DS_FstZS[xy_grid_cnt(v[p][0],v[p-1][0],ulX,v[p][1],v[p-1][1],ulY,clSze)])
    return int(total)

In [138]:
v = [(1608490,2038038),(1608450,2038038),(1608450,2038138),(1608490,2038138),(1608490,2038038)]

In [139]:
#g.shape
test = FstZS(v,DS_FstZS,ulX,ulY,csze)
test

1


ZeroDivisionError: division by zero

In [40]:
### Import Test vector
polygon = Polygon(v) 
zonal_stats(polygon,"data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif",stats= "sum")

C:\Users\smh362\AppData\Local\Continuum\Anaconda3\lib\site-packages\rasterstats\main.py:145: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  with Raster(raster, affine, nodata, band) as rast:
C:\Users\smh362\AppData\Local\Continuum\Anaconda3\lib\site-packages\rasterstats\io.py:242: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  self.affine = guard_transform(self.src.transform)


[{'sum': 246.0}]

Pretty Close Not sure why these are slightly different end